In [15]:
# Q1
numPartitions = 10

lines = sc.textFile('/Users/milkbread/adj_noun_pairs.txt', numPartitions)
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

# my code:
# pairs.collect()
# pairs.take(100) # 打印前100个元素
# pairs = sc.parallelize(pairs, 4).cache()
# 目前的思路是用forloop同时执行每个partition，然后把partition的结果合在一起比较
def f(iterator):
    for i in iterator:
        if i[1] == "unification":
            yield i
            break

SpecificNoun = pairs.mapPartitions(f).filter(lambda x: x[1] == "unification").first()

print SpecificNoun

(u'several', u'unification')


In [ ]:
sc.addPyFile('/Users/milkbread/graphframes-0.5.0-spark2.1-s_2.11.jar')

In [ ]:
# Q2
sc.addPyFile("../graphframes-0.5.0-spark2.1-s_2.11.jar")

from graphframes import *
from pyspark.sql.functions import *

# Vertics DataFrame
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 37),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 38),
  ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend"),
  ("g", "e", "follow")
], ["src", "dst", "relationship"])

# Create a GraphFrame
g = GraphFrame(v, e)

# g.vertices.show()
# g.edges.show()

motifs = g.find("(a)-[]->(b); (b)-[]->(c)")\
          .filter("a.name = 'Alice'")\
          .select("c.name") # edge不能相同
motifs.show()

In [ ]:
# Q3
motifs = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)")\
          .filter("a.name = 'Alice'")\
          .select("c.name") # edge不能相同
motifs.show()

In [ ]:
# Q4
# people = g.edges.filter("relationship = 'follow' AND dst = 'Charlie'")
# people.show()
peopele = e.join(v, e.src == v.id).filter("dst = 'c'and relationship = 'follow'").select("name")
people.show()

In [ ]:
# Q5
people = g.edges.filter("relationship = 'follow'").groupBy('dst').withColumnRenamed('dst', 'leader').count()
people.filter("count > 1").show()